## This script is used for the prepraring dataset for the finetuning DNABERT for SARS-CoV-2 virant clasiification 

In [53]:
import os, tarfile
import glob, time, csv, zipfile, gzip
import pandas as pd
import itertools as it
import sys

In [54]:
os.getcwd()

'/home/pratik/Work_directory/SARS_finetuned'

In [96]:
variant_dict = {
    "alpha": 0,
    "beta": 1,
    "delta": 2,
    "gamma": 3,
    "eta":4,
    "iota":5,
    "kappa":6,
    "lambda":7,
    "zeta":8
}

In [94]:
variant_dict['alpha']

0

In [55]:
#For Mac
#parent_dir_path = "/Users/pratik/Box/Datasets/work_dir/SARS_finetuned/"
#For Windows
parent_dir_path = "/home/pratik/Work_directory/SARS_finetuned/"
os.chdir(parent_dir_path)

### Function for creating non-overlappingg chunks with starting position zero and other mentioned starting position

In [177]:
def create_non_overlap_genome(sequence, chunk_size):
    chunks = []
    starting_position_list = [0, 1000, 2000]
    for position in starting_position_list:
        temp_chunks = [sequence[i:i+chunk_size] for i in range(position, len(sequence), chunk_size)]
        if(len(temp_chunks[-1])<chunk_size):
            del temp_chunks[-1]
        chunks.extend(temp_chunks)
    last_chunk = sequence[-chunk_size:]
    chunks.append(last_chunk)
    return(chunks)

### Finetuning data preparation

In [194]:
def finetune_data_preparation(file_lines, label):
    final_list = []
    for key,group in it.groupby(file_lines,lambda line: line.startswith('>')):
        if not key:
            #count= count+1
            group = list(group)
            stripped_group = list(map(lambda s: s.strip(), group))
            concat_group = ' '.join(stripped_group) 
            concat_group = concat_group.replace(" ","")
            temp_list= create_non_overlap_genome(concat_group, 3000)
            final_list.extend(temp_list)
    temp_df= pd.DataFrame(final_list, columns= ['Sequence'])
    print ("temp", temp_df.shape)
    temp_df = temp_df.drop_duplicates().reset_index(drop=True)
    temp_df['Label']= label
    return (temp_df)

## Remove contradict entries and save in a `.csv` file

In [213]:
def remv_contradict_entries(g, directory):
    print (g.shape)
    df1= g.groupby('Sequence')
    df2= df1.filter(lambda x: len(x) == 1).reset_index(drop=True)
    print (df2.groupby('Label').count())
    df2.to_csv("Finetuned_SARS_"+directory+".csv", index=False)

In [214]:
for directory in os.listdir():
    count =0
    print (directory)
    final_df= pd.DataFrame(columns= ['Sequence','Label' ])
    current_path= os.path.join(parent_dir_path,directory)
    #print (current_path)
    folders = os.listdir(directory)
    for subfolder in folders:
        path = os.path.join(current_path, subfolder)
        print (path)
        files= os.listdir(path)
        for file in files:
            if file.endswith("tar"):
                file_path= os.path.join(path, file)
                contents = tarfile.open(file_path)
                #print(contents.getnames())
                for file_name, file_object in zip(contents.getnames(), contents.getmembers()):
                    if file_name.endswith('fasta'):
                        print(file_name)
                        f=contents.extractfile(file_object)
                        file_content=f.readlines()
                        file_content_str = [x.decode() for x in file_content]
                        label = variant_dict[subfolder]
                        temp_df = finetune_data_preparation(file_content_str, label)
                        final_df = final_df.append(temp_df, ignore_index=True)
                        print ("Final",final_df.shape)
    remv_contradict_entries(final_df, directory)

Oceania
/home/pratik/Work_directory/SARS_finetuned/Oceania/beta
1629323731996.sequences.fasta
temp (1960, 1)
Final (1010, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/zeta
1629323703456.sequences.fasta
temp (140, 1)
Final (1128, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/eta
1629324106221.sequences.fasta
temp (196, 1)
Final (1268, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/lambda
1629324181209.sequences.fasta
temp (28, 1)
Final (1296, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/iota
1629324129472.sequences.fasta
temp (56, 1)
Final (1324, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/delta
1629323796416.sequences.fasta
temp (16100, 1)
Final (6334, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/gamma
1629323757740.sequences.fasta
temp (196, 1)
Final (6442, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/kappa
1629324155720.sequences.fasta
temp (2240, 1)
Final (7108, 2)
/home/pratik/Work_directory/SARS_finetuned/Oceania/alpha
1

In [206]:
test = final_df['Sequence']
len(test.drop_duplicates())

12656

In [ ]:
count =0
with open('/Users/pratik/Box/Datasets/20210712_SARS_COV_2_genome/sequences.fasta', 'r') as f:
    for line in f:
        if(line[0]=='>'):
            count = count+1
            if(count%100000==0):
                print(count)
print("Total number", count)

In [17]:
create_non_overlap_genome1('ASKNCLKENCEMCNVEFKLNHVSLKFNWLKF', 3, 2)

['KNC', 'LKE', 'NCE', 'MCN', 'VEF', 'KLN', 'HVS', 'LKF', 'NWL']

In [13]:
len(list_genome[0])

3000

In [ ]:
genome_set= {'AA','TTAT','AAT'}

In [ ]:
genome_set

In [ ]:
genome_set.add('AATT')

In [ ]:
genome_set

In [23]:
string= 'ASKNCLKENCEMCNVEFKLNHVSLKFNWLKF'
print(len(string))
chunk = 3

31


In [32]:
l = [string[i:i+3] for i in range(0, len(string), 3)]
print (l)

['ASK', 'NCL', 'KEN', 'CEM', 'CNV', 'EFK', 'LNH', 'VSL', 'KFN', 'WLK', 'F']


In [33]:
del l[-1]
l

['ASK', 'NCL', 'KEN', 'CEM', 'CNV', 'EFK', 'LNH', 'VSL', 'KFN', 'WLK']

In [25]:
if (len(l[-1])<3):
    last_element = string[-3:]
    del l[-1]
    l.append(last_element)
print(l)

['ASK', 'NCL', 'KEN', 'CEM', 'CNV', 'EFK', 'LNH', 'VSL', 'KFN', 'WLK', 'LKF']


In [26]:
l

['ASK', 'NCL', 'KEN', 'CEM', 'CNV', 'EFK', 'LNH', 'VSL', 'KFN', 'WLK', 'LKF']

In [ ]:
s= set()
s

In [ ]:
s.update(l)

In [ ]:
s

In [ ]:
l1= ['ASK', 'BBC']

In [ ]:
s.update(l1)

In [ ]:
s

In [ ]:
l

In [103]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Falcon', 'Parrot','Human'],
                   'Max Speed': [380., 370., 24.,370, 26., 1]})

In [104]:
df

,Animal,Max Speed
0,Falcon,380.0
1,Falcon,370.0
2,Parrot,24.0
3,Falcon,370.0
4,Parrot,26.0
5,Human,1.0


In [112]:
df.drop_duplicates().reset_index(drop=True)

,Animal,Max Speed
0,Falcon,380.0
1,Falcon,370.0
2,Parrot,24.0
3,Parrot,26.0
4,Human,1.0


In [119]:
g = df.groupby('Animal')
g.filter(lambda x: len(x) == 1).reset_index(drop=True)

,Animal,Max Speed
0,Human,1.0


In [122]:
l = [1,2,3,4,5,6]
df1= pd.DataFrame(l, columns=['Test'])
df1
df1['label']=0
df1

,Test,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0


In [ ]:
def list_to_multiple_csv(l):
    span = int(len(l)/3)+1
    list_of_sublist= [l[x:x+span] for x in range(0, len(l), span)]
    print (list_of_sublist)
    for i in range(len(list_of_sublist)):
        csv_file= "pretrain_SARS_COV_variant_"+str(i+1)+".txt"
        with open(csv_file, "w") as outfile:
            outfile.write("\n".join(list_of_sublist[i]))

In [ ]:
list_to_multiple_csv(l)

In [ ]:
                for member in contents.getmembers():
                    print (member)
                    f=contents.extractfile(member)
                    #print (f)
                    content=f.readlines()
                    #print (content)
                    print ("%s has %d newlines" %(member, content.count("\n")))
                    print ("%s has %d spaces" % (member,content.count(" ")))
                    print ("%s has %d characters" % (member, len(content)))
                input()
                index = {i.name: i for i in contents.getmembers()}
                print(index.keys())
                for element in index.keys():
                    if element.endswith("fasta"):
                        print(element)
                        fasta_file_path= os.path.join(file_path, element)
                        finetune_data_preparation(fasta_file_path)
                count =count+1